# **Análise de Sentimentos de _Reviews_ de produtos na plataforma Steam**

## **Importando bibliotecas**

In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np 
import re
import string
import unidecode

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipegruend/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Carregando _dataset_** 

In [3]:
data = pd.read_csv('dataset/steam_reviews.tsv', sep='\t')[['review', 'voted_up']]

In [4]:
data.describe()

,review,voted_up
count,507051,507052
unique,506693,2
top,Err:507,True
freq,226,460579


In [5]:
data.dtypes

review      object
voted_up      bool
dtype: object

In [6]:
data

,review,voted_up
0,"Corrupted é um jogo de RPG bem simplificado, o...",True
1,Meu PC tem uma placa de vídeo equivalente a GT...,False
2,Muito Bom! O meu diz que é recomendado specs:...,True
3,Funciona,True
4,"Muito bom o Steam VR, Very good.",True
...,...,...
507047,"Vamos lá, devido a falta de reviews em pt-br v...",True
507048,Joguei a versão v.0.85.1.1.753323 O jogo até o...,True
507049,"Consegui o que eu queria, então foda-se :D",True
507050,"Mao meno, jogo muito monotomo, para celular ir...",False


### **Separando Exemplos**

#### Selecionando apenas Exemplos Positivos

In [7]:
positive = data[data['voted_up'] == True].sample(46000)
positive

,review,voted_up
7731,"Jogo muito bom, jogabilidade exelente, enredo ...",True
392402,um otimo game mas para mim falta brasileiros n...,True
203058,"É um RPG antigo ,,muito bom,,",True
333981,"quem ja jogou e zerou o primeiro, concerterza ...",True
337751,T0P!,True
...,...,...
69996,"Jogo mto bom , é mto legal de jogar com amigos...",True
387345,WHATS,True
62081,Jogo muito bom. Esperando pelo próximo capítulo!,True
73822,Melhor jogo coop,True


#### Selecionando apenas Exemplos Negativos

In [8]:
negative = data[data['voted_up'] == False].sample(46000)
negative

,review,voted_up
325589,"nao recomendo, nao gosto e nao quero",False
251535,"O Jogo é muito bom , mas muuito bom que não qu...",False
293320,"A falta de servidores brasileiros dificultam, ...",False
125035,Na minha opnião ele parece mais um jogo de cel...,False
334750,"O jogo tinha potencial, porém virou um MoneyMa...",False
...,...,...
501949,tem muita queda de fps,False
360307,uma bosta :D,False
31295,"O jogo é bom e tem uma ótima premissa, mas as ...",False
261237,"Sem grafico e chato enjoante,reembolso salva E...",False


#### Concatenando os dados e misturando-os

In [9]:
frames = [positive, negative]
data = pd.concat(frames)

In [10]:
data = data.sample(frac=1).reset_index(drop=True)

In [11]:
data

,review,voted_up
0,A jogabilidade é muito travada de modo a torna...,False
1,"O lol dos mechs xD... muito divertido, pena qu...",True
2,"Não comprem, nem da pra jogar online.",False
3,"Um dos melhores jogos de Dragon Ball, valeu a ...",True
4,"O jogo tem uma ótima trilha sonora, e boa hist...",True
...,...,...
91995,"Tem um potencial imenso, mas... No lançamento ...",False
91996,eu me arrependo de ter jogando fora,True
91997,Achei que seria aquele tipo de passe que qualq...,False
91998,One of the best games of all time.,True


## **Processando os Dados**

In [12]:
def process_data(df):
    
    # To lowercase
    df['review'] = df['review'].str.lower()
    
    # Removing punctuations
    df['review'] = df['review'].str.replace('[^\w\s]','')
    
    # Removing all single characters
    df['review'] = df['review'].str.replace('\s+[a-zA-Z]\s+',' ')
    
    # Removing all single characters from the start
    df['review'] = df['review'].str.replace('\^[a-zA-Z]\s+',' ')
    
    # Substituting multiple spaces with single space
    df['review'] = df['review'].str.replace('\s+',' ')
    
    # Removing accents
    remove_accent = lambda x: unidecode.unidecode(x)
    df['review'] = df['review'].apply(remove_accent)
    
    return df

In [13]:
data = process_data(data)
data

,review,voted_up
0,a jogabilidade e muito travada de modo tornar ...,False
1,o lol dos mechs xd muito divertido pena que po...,True
2,nao comprem nem da pra jogar online,False
3,um dos melhores jogos de dragon ball valeu pen...,True
4,o jogo tem uma otima trilha sonora boa histori...,True
...,...,...
91995,tem um potencial imenso mas no lancamento do c...,False
91996,eu me arrependo de ter jogando fora,True
91997,achei que seria aquele tipo de passe que qualq...,False
91998,one of the best games of all time,True
